In [1]:
from astropy.coordinates import SkyCoord
from astropy.time import Time
from sbpy.data import Ephem
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn
seaborn.set_context('poster')

In [2]:
traj = pd.read_parquet("~/fink_fat_out/candidates_15_2_2/trajectory_orb.parquet")
orb = pd.read_parquet("~/fink_fat_out/candidates_15_2_2/orbital.parquet")

In [3]:
id_follow_up = np.array(
    [
        'FF2023aaaaalx', # FF2022aaaaawi, 57262
        'FF2023aaaaama', # FF2022aaaaawm, 57392
        'FF2023aaaaamb', # FF2022aaaaawn, 57417
        'FF2023aaaaamc', # FF2022aaaaawo, 57529
        'FF2023aaaaamd', # FF2022aaaaawp, 57438
    ], 
)

In [4]:
orb[orb['ssoCandId'].isin(id_follow_up)]

,ref_epoch,a,e,i,long. node,arg. peric,mean anomaly,rms_a,rms_e,rms_i,rms_long. node,rms_arg. peric,rms_mean anomaly,chi_reduced,ssoCandId
0,2.459814e+06,50.430875,0.926643,2.796635,137.321276,168.058791,0.926061,-1.000000,-1.000000,-1.000000,-1.00000,-1.00000,-1.0000,-1.000000,FF2023aaaaalx
0,2.459820e+06,8.085766,0.404250,4.198385,161.458208,216.409921,352.731526,0.830293,0.228862,0.612313,2.75339,313.97800,121.8200,0.221667,FF2023aaaaama
2,2.459820e+06,6.657587,0.337133,2.500486,130.961333,233.292874,358.370126,9.591750,1.052250,0.824765,18.63300,775.92300,374.0360,0.361667,FF2023aaaaamb
0,2.459821e+06,2.358976,0.251121,5.275541,304.750345,58.157846,342.815164,0.081354,0.008456,0.377315,1.06359,2.43947,1.2571,2.728333,FF2023aaaaamc
0,2.459823e+06,6.525971,0.783301,4.540030,173.979493,319.426800,326.989644,-1.000000,-1.000000,-1.000000,-1.00000,-1.00000,-1.0000,-1.000000,FF2023aaaaamd


In [34]:
print('ssoCandId, npoint, lastdate, arc length')
for id_ in id_follow_up:
    sub = traj[traj['ssoCandId'] == id_]
    ld = sub[sub['jd'] == np.max(sub['jd'])]['last_assoc_date'].values[0]
    arc = np.max(sub['jd']) - np.min(sub['jd'])
    print(id_, len(sub), ld, arc)

ssoCandId, npoint, lastdate, arc length
FF2023aaaaalx 6 2022-08-22 11.96789349988103
FF2023aaaaama 6 2022-08-28 6.011331100016832
FF2023aaaaamb 6 2022-08-28 4.06982639990747
FF2023aaaaamc 6 2022-08-29 7.989247700199485
FF2023aaaaamd 6 2022-08-31 9.134259299840778


## Utils

In [6]:
def compute_sep_from_ztf(traj, fink_id, mpc_id, id_type=None):
    """ Compute the separation between a Fink trajectory and a SSO object
        
    Parameters
    ----------
    fink_id: str
        Fink-FAT ssoCandId
    mpc_id: str or int
        MPC number or designation
    id_type: None or str
        If the mpc_id is a major body (e.g. planet satellite), specify id_type="id"
        
    Returns
    -----------
    out: list
        List of separation between the Fink-FAT trajectory alerts and the SSO
    """
    t_ztf = traj[traj['ssoCandId'] == fink_id]['jd'].values
    epoch = Time(t_ztf, format='jd', scale='utc') # time in UT
    
    if id_type is None:
        eph = Ephem.from_horizons(mpc_id, location='I41', epochs=epoch)
    else:
        eph = Ephem.from_horizons(mpc_id, id_type=id_type, location='I41', epochs=epoch)

    t = eph.table
    prediction = SkyCoord(t['RA'], t['DEC'])

    obs = SkyCoord(
        traj[traj['ssoCandId'] == fink_id]['ra'].values,
        traj[traj['ssoCandId'] == fink_id]['dec'].values,
        unit='deg'
    )

    return prediction.separation(obs)

def deviation_from_single(ra_pred, dec_pred, t_pred, mpc_id, location='W85', id_type=None):
    """ Compute the deviation between a prediction/observation and a SSO object
        
    Parameters
    ----------
    ra_pred: str
        RA in whatever unit
    dec_pred: str
        DEC in whatever unit
    t_pred: str
        Time of the prediction (iso)
    mpc_id: str or int
        MPC number or designation
    id_type: None or str
        If the mpc_id is a major body (e.g. planet satellite), specify id_type="id"
        
    Returns
    -----------
    out: astropy angle
        Separation between the prediction and the SSO
    """
    epoch = Time(t_pred, scale='utc') # time in UT
    
    if id_type is None:
        eph = Ephem.from_horizons(mpc_id, location=location, epochs=epoch)
    else:
        eph = Ephem.from_horizons(mpc_id, id_type=id_type, location='I41', epochs=epoch)

    t = eph.table
    prediction = SkyCoord(t['RA'], t['DEC'])

    obs = SkyCoord(
        ra_pred, dec_pred, unit='deg'
    )

    return prediction.separation(obs)

## FF2023aaaaalx/FF2022aaaaawi/57262

This one was followed up during the LCOGT campaign (observation date on 2022/10/01). William reported the following follow-up observations

```
     AstXXXX  C2022 10  1.09963 00 19 30.34 -00 43 07.0          17.9 r      W85
     AstXXXX  C2022 10  1.10260 00 19 30.26 -00 43 07.6          17.9 r      W85
     AstXXXX  C2022 10  1.10557 00 19 30.18 -00 43 08.1          17.9 r      W85
     AstXXXX  C2022 10  1.10856 00 19 30.10 -00 43 08.7          18.0 r      W85
     AstXXXX  C2022 10  1.11152 00 19 30.01 -00 43 09.2          17.9 r      W85
     AstXXXX  C2022 10  1.11449 00 19 29.93 -00 43 09.8          18.0 r      W85
```

And the following asteroid associations:

```
(152295) 2005 TX42       00 19 22.7 -00 41 50  18.1   1.9W   1.3N    41-     0-   18o  None needed at this time.
(425019) 2009 EK29       00 19 26.6 -00 39 22  20.6   0.9W   3.8N    25-    18-   11o  None needed at this time.
```

In addition he checked for Jupiter moon and found:

```
v5.22.02i     Planet: Jupiter CALCEPH:  (INPOP19a)
        Satellite: J9 Sinope  ( Emelyanov N.V. Version 2021-01-31)
                               - ephemerides on 1905/01/01.0-2050/02/09.0
    Observatory N: W87 - Cerro Tololo-LCO C
        Timescale: UTC
Mean equator and equinox of J2000.  ICRF. 
Coordinates astrometric 
Year Month  Day     alpha (h m s)      delta (º ' ")
2022 10  1.099630  0 19 30.343416    0 -43   7.005888
```

So we have 3 objects nearby to check. We will compute the distance between the objects found and the ZTF observations (association in the past), at the time of the ZTF observations to see if those candidate objects are still relevant:

### (152295) 2005 TX42

In [7]:
d = deviation_from_single('00h19m29.93s', '-00d43m09.8s', '2022-10-01 02:00:00', mpc_id=152295, location='W85')
print('Distance between the object and William observation: ', d[0])

print()
print('Distance between the object and ZTF observation at the time of the trajectory: ')
compute_sep_from_ztf(traj, 'FF2023aaaaalx', 152295)

Distance between the object and William observation:  0d02m03.1166s

Distance between the object and ZTF observation at the time of the trajectory: 


/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'surfbright': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'illum_defect': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'ang_width': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'PDObsLon': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: 

<Angle [4.55711024, 4.55724255, 4.57363912, 4.57807693, 4.50245758,
        4.50082045] deg>

### (425019) 2009 EK29

In [8]:
print(deviation_from_single('00h19m29.93s', '-00d43m09.8s', '2022-10-01 02:00:00', 425019, location='W85'))

compute_sep_from_ztf(traj, 'FF2023aaaaalx', 425019)

[0d03m58.4296s]


<Angle [3.94495749, 3.94357474, 3.87078016, 3.65125925, 3.39051323,
        3.38758776] deg>

### Jupiter IX (Sinope)

In [9]:
print(deviation_from_single('00h19m29.93s', '-00d43m09.8s', '2022-10-01 02:00:00', 'JIX', location='W85', id_type='id'))

compute_sep_from_ztf(traj, 'FF2023aaaaalx', 'Sinope', id_type='id')

[0d00m14.0438s]


/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'RA_3sigma': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'DEC_3sigma': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'SMAA_3sigma': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'SMIA_3sigma': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning

<Angle [2.92384030e-05, 2.20027084e-05, 8.01482423e-06, 3.27534357e-05,
        2.52180334e-05, 1.42003520e-05] deg>

We can also check in this website: https://www.minorplanetcenter.net/iau/NatSats/NaturalSatellites.html

In [10]:
jds = traj[traj['ssoCandId'] == 'FF2023aaaaalx']['jd'].values 
mask = jds == np.max(jds)

Time(jds[mask][0], format='jd').iso

'2022-08-22 09:17:46.997'

The closest ephemerides in time gives (read from the website):

In [11]:
ra_eph = '00h34m20.9s'
dec_eph = '+01d10m31s'

eph = SkyCoord(ra_eph, dec_eph)

ra_ztf = traj[traj['ssoCandId'] == 'FF2023aaaaalx']['ra'].values[mask][0]
dec_ztf = traj[traj['ssoCandId'] == 'FF2023aaaaalx']['dec'].values[mask][0]

eph_ztf = SkyCoord(ra_ztf, dec_ztf, unit='deg')

eph.separation(eph_ztf)

<Angle 0.00044837 deg>

### Conclusion

At the time of ZTF observations, only Jupiter IX was compatible :-)

### Deviation

the observation was made 1 month after the last prediction -- let compute the 

In [12]:
# taken from Roman's ephemerides 2022-09-30
ra_pred, dec_pred = 4.958566717023006,-0.6869169649257217
deviation_from_single(ra_pred, dec_pred, '2022-09-30 23:30:00', mpc_id='JIX', id_type='id')

/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'magextinct': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))


<Angle [0.07317127] deg>

## FF2023aaaaama/FF2022aaaaawm/57392

This one was followed up during the LCOGT campaign (observation date on 2022/10/01). William reported the following follow-up observations

```
     AstXXXX  C2022 10  1.08975 00 09 42.29 -01 02 40.7          16.5 r      W87
     AstXXXX  C2022 10  1.09162 00 09 42.24 -01 02 41.1          16.7 r      W87
     AstXXXX  C2022 10  1.09354 00 09 42.20 -01 02 41.4          16.6 r      W87
     AstXXXX  C2022 10  1.09650 00 09 42.13 -01 02 41.9          16.6 r      W87
     AstXXXX  C2022 10  1.09946 00 09 42.05 -01 02 42.5          16.6 r      W87
     AstXXXX  C2022 10  1.10244 00 09 41.99 -01 02 43.0          16.6 r      W87
     AstXXXX  C2022 10  1.10539 00 09 41.91 -01 02 43.5          16.6 r      W87
     AstXXXX  C2022 10  1.10769 00 09 41.85 -01 02 43.9          16.6 r      W87
     AstXXXX  C2022 10  1.14189 00 09 41.03 -01 02 50.2          16.5 r      W87
     AstXXXX  C2022 10  1.14486 00 09 40.95 -01 02 50.8          16.5 r      W87
     AstXXXX  C2022 10  1.14784 00 09 40.88 -01 02 51.3          16.5 r      W87
     AstXXXX  C2022 10  1.15080 00 09 40.81 -01 02 51.8          16.6 r      W87
     AstXXXX  C2022 10  1.15376 00 09 40.73 -01 02 52.4          16.5 r      W87
     AstXXXX  C2022 10  1.15673 00 09 40.66 -01 02 52.9          16.6 r      W87
```

And the following asteroid associations (far field):

```
         2012 XF166      00 09 54.0 -01 00 57  21.6   3.0E   1.8N    33-     3-    5o 
(549752) 2011 SQ146      00 09 52.9 -00 59 49  19.8   2.8E   2.9N    33-     9-   10o 
```

In addition he checked for Jupiter moon and found:

```
Verification pour J8 Pasiphae

Je regarde d'abord sur https://ssp.imcce.fr/forms/relative-positions  dans l onglet "configuration"
quel pourrait etre le satellite de Jupiter proche de cet objet et de meme magnitude:  peut-etre J8 Pasiphae

Ensuite je vérifie plus exactement sur: http://nsdb.imcce.fr/multisat/nssre10he.htm
avec la premiere date des mesures ci-dessus

5.22.02i     Planet: Jupiter CALCEPH:  (INPOP19a)
        Satellite: J8 Pasiphae  ( Emelyanov N.V. Version 2021-01-31)
                               - ephemerides on 1905/01/01.0-2050/02/09.0
    Observatory N: W87 - Cerro Tololo-LCO C
        Timescale: UTC
Mean equator and equinox of J2000.  ICRF. 
Coordinates astrometric 
Year Month  Day     alpha (h m s)      delta (º ' ")
2022 10  1.089750  0  9 42.278580   -1   2  40.726002

Pile dessus!  ==> donc c'est bien J8 Pasiphae
```

So we have 3 objects nearby to check. We will compute the distance between the objects found and the ZTF observations (association in the past), at the time of the ZTF observations to see if those candidate objects are still relevant:

In [13]:
print(deviation_from_single('00h09m40.66s', '-01d02m52.9s', '2022-10-01 02:00:00', mpc_id='2012 XF166'))


compute_sep_from_ztf(traj, 'FF2023aaaaama', '2012 XF166')

[0d04m08.1847s]


<Angle [4.0140301 , 4.01333554, 3.88241214, 3.74391588, 3.74205967,
        3.59434232] deg>

In [14]:
print(deviation_from_single('00h09m40.66s', '-01d02m52.9s', '2022-10-01 02:00:00', mpc_id=549752))

compute_sep_from_ztf(traj, 'FF2023aaaaama', 549752)

[0d04m37.0121s]


<Angle [2.87247631, 2.87268576, 2.91702067, 2.93674875, 2.9366123 ,
        2.93141699] deg>

In [15]:
compute_sep_from_ztf(traj, 'FF2023aaaaama', 'JVIII', id_type='id')

<Angle [4.15453119e-05, 3.99230898e-05, 1.41591385e-05, 4.75106186e-05,
        3.72220636e-05, 4.51059998e-05] deg>

Jupiter VIII was the closest during ZTF observations!

### Deviation

In [16]:
# taken from Roman's ephemerides 2022-09-30
ra_pred, dec_pred = 2.4326186183052916,-1.049377010400635
deviation_from_single(ra_pred, dec_pred, '2022-09-30 23:30:00', mpc_id='JVIII', id_type='id')

<Angle [0.0106888] deg>

## FF2023aaaaamb/FF2022aaaaawn/57417

This one was followed up during the LCOGT campaign (observation date on 2022/09/25). William reported the following follow-up observations

```
     AstXXXX  C2022 09 25.24131 00 22 09.45 -00 24 43.0          17.6 r      W85
     AstXXXX  C2022 09 25.24450 00 22 09.36 -00 24 43.6          17.6 r      W85
     AstXXXX  C2022 09 25.24770 00 22 09.27 -00 24 44.2          17.6 r      W85
     AstXXXX  C2022 09 25.25091 00 22 09.18 -00 24 44.9          17.6 r      W85
     AstXXXX  C2022 09 25.25410 00 22 09.09 -00 24 45.4          17.6 r      W85
     AstXXXX  C2022 09 25.25730 00 22 09.01 -00 24 46.1          17.6 r      W85
```

And the following asteroid associations (far field):

```
(426612) 2013 SY46       00 22 10.6 -00 22 54  20.4   0.3E   1.8N    31-    15-   16o  
(274218) 2008 JU30       00 22 06.3 -00 28 24  19.8   0.8W   3.7S    32-    29-   10o  
```

In addition he checked for Jupiter moon and found:

```
http://nsdb.imcce.fr/multisat/nsso-10he.htm 
donne:
v5.22.02i     Planet: Jupiter CALCEPH:  (INPOP19a)
        Satellite: J9 Sinope  ( Emelyanov N.V. Version 2021-01-31)
                               - ephemerides on 1905/01/01.0-2050/02/09.0
    Observatory N: W87 - Cerro Tololo-LCO C
        Timescale: UTC
Mean equator and equinox of J2000.  ICRF. 
Coordinates astrometric 
Year Month  Day     alpha (h m s)      delta (º ' ")
2022  9 25.241310  0 22  9.453143    0 -24  42.991085
```

So we have 3 objects nearby to check. We will compute the distance between the objects found and the ZTF observations (association in the past), at the time of the ZTF observations to see if those candidate objects are still relevant:

In [17]:
print(deviation_from_single('00h22m09.01s', '-00d24m46.1s', '2022-09-25 06:00:00', mpc_id=426612))

compute_sep_from_ztf(traj, 'FF2023aaaaamb', 426612)

[0d01m49.9464s]


<Angle [2.27161984, 2.27093865, 2.22129892, 2.2213349 , 2.22053207,
        2.15236171] deg>

In [18]:
print(deviation_from_single('00h22m09.01s', '-00d24m46.1s', '2022-09-25 06:00:00', mpc_id=274218))

compute_sep_from_ztf(traj, 'FF2023aaaaamb', 274218)

[0d03m49.6373s]


<Angle [4.29713602, 4.29516937, 4.14739868, 4.14744758, 4.14537885,
        3.96895281] deg>

In [19]:
compute_sep_from_ztf(traj, 'FF2023aaaaamb', 'JIX', id_type='id')

<Angle [8.68131236e-06, 2.28540784e-05, 1.66000276e-05, 1.35236753e-05,
        1.41989012e-05, 2.50279084e-05] deg>

### Deviation

In [20]:
# taken from Roman's ephemerides 2022-09-30
ra_pred, dec_pred = 4.885296234580493,-0.714489354518262
deviation_from_single(ra_pred, dec_pred, '2022-09-30 23:30:00', mpc_id='JIX', id_type='id')

<Angle [0.00512667] deg>

### Note

1. William found however 4 new objects in the exposures during the observation (unrelated to what we sent). These observations were sent to the MPC.
2. This object was also observed at OHP, and confirmed Sinope.

## FF2023aaaaamc/FF2022aaaaawo/57529

This one was followed up during the LCOGT campaign (observation date on 2022/10/01). William reported the following follow-up observations

```
d4919         C2022 10  1.04864 21 37 38.99 -05 26 39.1          19.1 r      W87
d4919         C2022 10  1.05162 21 37 38.96 -05 26 39.6          18.5 r      W87
d4919         C2022 10  1.05459 21 37 38.88 -05 26 40.6          18.7 r      W87
d4919         C2022 10  1.05755 21 37 38.79 -05 26 41.2          19.5 r      W87
d4919         C2022 10  1.06052 21 37 38.82 -05 26 41.6          19.7 r      W87
```

with the comment `connu`, and the following asteroid associations:

```
(394919) 2008 WN27       21 37 38.9 -05 26 39  21.4   0.0W   0.0N     7-    10-   10o   
```

So we have 1 object nearby to check. We will compute the distance between the object found and the ZTF observations (association in the past), at the time of the ZTF observations to see if this candidate object is still relevant:

In [21]:
print(deviation_from_single('21h37m38.82s', '-05d26m41.6s', '2022-10-01 01:00:00', mpc_id=394919))


compute_sep_from_ztf(traj, 'FF2023aaaaamc', 394919)

[0d00m05.2941s]


<Angle [4.35502646, 4.35142709, 4.19533114, 4.03048324, 4.02612775,
        3.70215457] deg>

### Conclusion

This object has no counterpart.

## FF2023aaaaamd/FF2022aaaaawp/57438

This one was followed up during the LCOGT campaign (observation date on 2022/10/01). William reported the following follow-up observations

```
a3563         C2022 10  1.07493 00 10 16.70 +00 25 00.9          19.8 r      W85
a3563         C2022 10  1.07790 00 10 16.53 +00 25 00.5          19.7 r      W85
a3563         C2022 10  1.08087 00 10 16.36 +00 24 59.9          19.8 r      W85
a3563         C2022 10  1.08385 00 10 16.16 +00 24 59.4          19.8 r      W85
a3563         C2022 10  1.08682 00 10 15.98 +00 24 58.9          19.6 r      W85
a3563         C2022 10  1.08979 00 10 15.80 +00 24 58.5          19.6 r      W85
(363563) 2003 YW27       00 10 16.8 +00 25 01  19.9   0.0E   0.0N    38-     6-    9o  

h5953         C2022 10  1.07493 00 10 19.76 +00 19 09.4          20.9 r      W85
h5953         C2022 10  1.08682 00 10 19.19 +00 19 05.9          21.2 r      W85
h5953         C2022 10  1.08979 00 10 19.06 +00 19 04.9          20.2 r      W85
(435953) 2009 DM20       00 10 19.8 +00 19 10  21.3   0.0E   0.0N    29-    13-    8o  

connus:

X9694         C2022 10  1.07493 00 10 11.44 +00 12 27.8          17.1 r      W85
X9694         C2022 10  1.07790 00 10 11.29 +00 12 27.5          16.9 r      W85
X9694         C2022 10  1.08087 00 10 11.13 +00 12 27.2          17.2 r      W85
X9694         C2022 10  1.08385 00 10 10.96 +00 12 26.4          17.3 r      W85
X9694         C2022 10  1.08682 00 10 10.80 +00 12 25.1          17.1 r      W85
X9694         C2022 10  1.08979 00 10 10.63 +00 12 23.9          17.1 r      W85
(339694) 2005 QU186      00 10 11.5 +00 12 32  19.6   0.0E   0.1N    34-    17-   10o  

52703         C2022 10  1.07493 00 09 50.41 +00 24 11.4          18.9 r      W85
52703         C2022 10  1.07790 00 09 50.28 +00 24 10.5          18.9 r      W85
52703         C2022 10  1.08087 00 09 50.14 +00 24 09.7          19.0 r      W85
52703         C2022 10  1.08385 00 09 49.99 +00 24 08.8          19.0 r      W85
52703         C2022 10  1.08682 00 09 49.87 +00 24 08.0          19.0 r      W85
52703         C2022 10  1.08979 00 09 49.73 +00 24 07.2          19.0 r      W85
 (52703) 1998 FW72       00 09 50.5 +00 24 12  19.1   0.0E   0.0N    28-    11-   21o 

     K15Bj1H  C2022 10  1.08087 00 08 46.48 +00 30 36.1          20.7 r      W85
     K15Bj1H  C2022 10  1.08682 00 08 46.16 +00 30 34.2          21.3 r      W85
     K15Bj1H  C2022 10  1.08979 00 08 45.99 +00 30 33.7          20.8 r      W85
         2015 BH451      00 08 46.5 +00 30 36  20.6   0.0E   0.0S    34-     9-    7o 
```

So we have 1 object nearby to check. We will compute the distance between the object found and the ZTF observations (association in the past), at the time of the ZTF observations to see if this candidate object is still relevant:

In [22]:
print(deviation_from_single('00h10m15.80s', '+00d24m58.5s', '2022-10-01 02:00:00', mpc_id=363563))


compute_sep_from_ztf(traj, 'FF2023aaaaamd', 363563)

[0d00m06.0254s]


<Angle [3.59311059, 3.56992936, 3.53289229, 3.53219989, 3.37115893,
        3.36777524] deg>

In [23]:
print(deviation_from_single('00h10m19.06s', '+00d19m04.9s', '2022-10-01 02:00:00', mpc_id=435953))


compute_sep_from_ztf(traj, 'FF2023aaaaamd', 435953)

[0d00m04.8547s]


<Angle [2.1067372 , 2.1324288 , 2.14522842, 2.14520794, 2.12459669,
        2.12352174] deg>

In [24]:
print(deviation_from_single('00h10m10.63s', '+00d12m23.9s', '2022-10-01 02:00:00', mpc_id=339694))


compute_sep_from_ztf(traj, 'FF2023aaaaamd', 339694)

[0d00m06.6096s]


<Angle [3.29993274, 3.35406265, 3.37964799, 3.37957562, 3.32710677,
        3.32455122] deg>

In [25]:
print(deviation_from_single('00h09m49.73s', '+00d24m07.2s', '2022-10-01 02:00:00', mpc_id=52703))


compute_sep_from_ztf(traj, 'FF2023aaaaamd', 52703)

[0d00m04.8389s]


<Angle [1.92199301, 1.92888014, 1.92663274, 1.9264686 , 1.88176669,
        1.88054528] deg>

In [26]:
print(deviation_from_single('00h08m45.99s', '+00d30m33.7s', '2022-10-01 02:00:00', mpc_id='2015 BH451'))


compute_sep_from_ztf(traj, 'FF2023aaaaamd', '2015 BH451')

[0d00m05.5447s]


<Angle [2.07392227, 2.11742836, 2.14517836, 2.14526408, 2.14793199,
        2.14686452] deg>

### Conclusion

No counterpart.

## Check

Check on known asteroid that our function returns small deviations

In [27]:
skybot = pd.read_parquet('results_analysis/skybot_result.parquet')

In [28]:
for index, i in enumerate(skybot.head(40)['d:candid']):
    if not traj[traj['candid'] == i].empty:
        id_ = traj[traj['candid'] == i]['ssoCandId'].values[0]
        sky_name = skybot['Ast_Name'].values[index]
        if sky_name != 'null':
            print(id_, sky_name)
            print('deviation: ', compute_sep_from_ztf(traj, id_, str(sky_name)))

FF2023aaaaaaz 2001 QX335
deviation:  [0d00m00.2952s 0d00m00.2978s 0d00m00.1079s 0d00m00.1808s 0d00m00.093s
 0d00m00.1783s 0d00m00.2773s]
FF2023aaaaagl 1996 AS13
deviation:  [0d00m00.3905s 0d00m00.3271s 0d00m00.3164s 0d00m00.2184s 0d00m00.1103s
 0d00m00.264s]
FF2023aaaaabn 2016 AN313
deviation:  [0d00m00.2528s 0d00m00.0811s 0d00m00.1652s 0d00m00.3358s 0d00m00.3924s
 0d05m57.083s]
FF2023aaaaajl 2000 SJ156
deviation:  [0d00m00.0953s 0d00m00.0533s 0d00m00.2293s 0d00m00.3674s 0d07m56.0692s
 0d07m08.4797s 0d07m07.5489s]
FF2023aaaaafq 2020 TB24
deviation:  [0d00m00.3732s 0d00m00.3119s 0d00m00.1888s 0d00m00.1737s 0d00m00.1439s
 0d00m00.3017s]
FF2023aaaaaaf 2019 RV4
deviation:  [0d00m00.139s 0d00m00.1781s 0d00m00.1996s 0d00m00.1684s 0d00m00.1344s
 0d00m00.2439s]
FF2023aaaaaaq 2013 SA105
deviation:  [0d00m00.2182s 0d00m00.3094s 0d00m00.1436s 0d00m00.1778s 0d00m00.0804s
 0d36m23.8669s]
FF2023aaaaafw 2006 UD221
deviation:  [0d00m00.2013s 0d00m00.2712s 0d00m00.4599s 0d00m00.2606s 0d00m00.1821s
 0d0

## Notes

Telescope features: https://noirlab.edu/public/programs/ctio/lco/1m-telescope4/

## First campaign

In [29]:
traj2 = pd.read_parquet("~/fink_fat_out/candidates1/trajectory_orb.parquet")
orb2 = pd.read_parquet("~/fink_fat_out/candidates1/orbital.parquet")

In [30]:
orb2[orb2['ssoCandId'] == 'FF2022aaaaamr']

,ref_epoch,a,e,i,long. node,arg. peric,mean anomaly,rms_a,rms_e,rms_i,rms_long. node,rms_arg. peric,rms_mean anomaly,chi_reduced,ssoCandId
0,2.459357e+06,3.055472,0.225826,16.678505,36.265627,162.942721,15.478685,0.024072,0.003458,0.371141,0.014758,4.76627,2.84111,2.34,FF2022aaaaamr


In [31]:
compute_sep_from_ztf(traj2, 'FF2022aaaaamr', 525570)

/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'surfbright': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'illum_defect': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'ang_width': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: dropping mask in Quantity column 'PDObsLon': masked Quantity not supported
  "masked Quantity not supported".format(col.info.name))
/opt/Anaconda3-2020.02/lib/python3.7/site-packages/astropy/table/table.py:3487: UserWarning: 

<Angle [1.75329878e-05, 4.39543606e-05, 3.61513454e-05, 5.80387739e-05,
        5.85307567e-05, 9.36047204e-05] deg>

In [32]:
# taken from Roman's ephemerides 2022-07-05
ra_pred, dec_pred = 340.32380967258354,-31.769628673858765
deviation_from_single(ra_pred, dec_pred, '2022-07-05 12:00:00', mpc_id=525570)

<Angle [0.14888182] deg>